In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from planner import *
import logging
from copy import copy
from enum import Enum

import numpy as np
from pydrake.all import (
    AbstractValue,
    AddMultibodyPlantSceneGraph,
    Concatenate,
    DiagramBuilder,
    InputPortIndex,
    LeafSystem,
    MeshcatVisualizer,
    Parser,
    PiecewisePolynomial,
    PiecewisePose,
    PointCloud,
    PortSwitch,
    RandomGenerator,
    RigidTransform,
    RollPitchYaw,
    Simulator,
    StartMeshcat,
    UniformlyRandomRotationMatrix,
    PortSwitch
)

from manipulation import ConfigureParser, FindResource, running_as_notebook
from manipulation.clutter import GenerateAntipodalGraspCandidate
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.pick import (
    MakeGripperCommandTrajectory,
    MakeGripperFrames,
    MakeGripperPoseTrajectory,
)
from manipulation.scenarios import AddIiwaDifferentialIK, ycb
from manipulation.station import (
    AddPointClouds,
    MakeHardwareStation,
    add_directives,
    load_scenario,
)

from IPython.display import SVG, display

class NoDiffIKWarnings(logging.Filter):
    def filter(self, record):
        return not record.getMessage().startswith("Differential IK")


logging.getLogger("drake").addFilter(NoDiffIKWarnings())
import pydot

In [3]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7003


In [4]:
import os
import pydot
import numpy as np
from pydrake.all import (
    Simulator,
    DiagramBuilder,
    StartMeshcat,
    MeshcatVisualizer,
    MeshcatVisualizerParams
)
from IPython.display import SVG, display
from manipulation.station import MakeHardwareStation, load_scenario

def init_scenario(brick_location=None, brick_rotation=None, meshcat=None):
    if meshcat is None:
        meshcat = StartMeshcat()
    
    if brick_location is None:
        table_xmin = 0.55
        table_xmax = 0.75
        table_ymin = -0.3
        table_ymax = 0.3
        x = np.random.uniform(table_xmin, table_xmax)
        y = np.random.uniform(table_ymin, table_ymax)
        z = 0
        brick_location = [x, y, z] 

    if brick_rotation is None:
        brick_rotation = [0, 0, np.random.randint(0, 360)]

    allegro_hand = "../gripper_sdfs/allegro_hand_description_right.sdf"

    gripper = os.path.join(os.getcwd(), allegro_hand)
    foam_brick = os.path.join(os.getcwd(), "../drake_hydro_sdfs/061_foam_brick.sdf")
    table_top = os.path.join(os.getcwd(), "../drake_hydro_sdfs/extra_heavy_duty_table_surface_only_collision.sdf")
    scenario_data = """
        directives:
        - add_model:
            name: iiwa
            file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
            default_joint_positions:
                iiwa_joint_1: [0]
                iiwa_joint_2: [0.1]
                iiwa_joint_3: [0]
                iiwa_joint_4: [-1.5]
                iiwa_joint_5: [0]
                iiwa_joint_6: [0]
                iiwa_joint_7: [0]
        - add_weld:
            parent: world
            child: iiwa::iiwa_link_0
            X_PC:
                translation: [-0.1, 0, 0]
                rotation: !Rpy { deg: [0, 0, 0]}
        - add_model:
            name: allegro
            file: file://""" + gripper + """
        - add_weld:
            parent: iiwa::iiwa_link_7
            child: allegro::hand_root
            X_PC:
                translation: [0, 0, 0.05]
                rotation: !Rpy { deg: [0, -90, 0]}
        # And now the environment:
        - add_model:
            name: foam_brick
            file: file://""" + foam_brick + """
            default_free_body_pose:
                base_link:
                    translation: """ + str(brick_location) + """
                    rotation: !Rpy { deg: """ + str(brick_rotation) + """}
        - add_model:
            name: robot_table
            file: file://""" + table_top + """
        - add_weld:
            parent: world
            child: robot_table::link
            X_PC:
                translation: [-0.1, 0, -0.7645]
        - add_model:
            name: work_table
            file: file://""" + table_top + """
        - add_weld:
            parent: world
            child: work_table::link
            X_PC:
                translation: [0.75, 0, -0.7645]
        model_drivers:
            iiwa+allegro: !InverseDynamicsDriver {}
        """
    builder = DiagramBuilder()
    scenario = load_scenario(data=scenario_data)
    station = builder.AddSystem(MakeHardwareStation(scenario, meshcat))

    return builder, station, scenario


def render_station_diagram(station):
    display(
    SVG(
        pydot.graph_from_dot_data(station.GetGraphvizString(max_depth=1))[
            0
        ].create_svg()
    )
    )

In [11]:
builder, station, scenario = init_scenario()
plant = station.GetSubsystemByName("plant")
planner = builder.AddSystem(Planner(plant))

builder.Connect(station.GetOutputPort("body_poses"),
                planner.GetInputPort("body_poses"))
builder.Connect(station.GetOutputPort("iiwa+allegro.state_estimated"),
                planner.GetInputPort("iiwa+allegro.state_estimated"))
# builder.Connect(station.GetOutputPort("contact_results"),
#                 planner.GetInputPort("contact_results"))

Diff_IK = builder.AddSystem(Differential_IK(plant))
builder.Connect(station.GetOutputPort("iiwa+allegro.state_estimated"),
                Diff_IK.GetInputPort("iiwa+allegro.state_estimated"))
builder.Connect(station.GetOutputPort("body_poses"),
                Diff_IK.GetInputPort("body_poses"))
builder.Connect(planner.GetOutputPort("X_WG"),
                Diff_IK.GetInputPort("X_WG_future"))

builder.Connect(
    Diff_IK.GetOutputPort("iiwa+allegro.desired_state"),
    station.GetInputPort("iiwa+allegro.desired_state")
)
diagram = builder.Build()
simulator = Simulator(diagram)
context = simulator.get_context()
plant_context = plant.GetMyMutableContextFromRoot(context)
simulator.AdvanceTo(0.5)

INFO:drake:Meshcat listening for connections at http://localhost:7010


KeyboardInterrupt: 